# Collect Data from the Twitter Search API, store them and preprocess them
### 1. Collect data from the Twitter Search API
##### Cause twitter only return data from the past 7 days, I collect data everyday for the recent tweets. 
##### 100 tweets per .json file
#####  The keywords I use is mainly about the recent UA3411 incident
### 2. Storing Data
##### To make sure there is no data overwrite, I saved each .json file with its query and an unix timestamp
##### I catagorize the .json files in folders named by its keywords and they will be catagorized by time
### 3. Preprocess Data
##### Extract all the tweets I collect from the raw data and store them as a list in a .json file
##### Get the information of users info, tweets, location, country and other data I will use in the future analysis and store them in a .csv file

In [8]:
import json
import requests
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream
import time
import datetime
from datetime import timedelta, date
import os
import glob
import shutil
import pandas as pd

### 1. Collect data from the Twitter Search API

In [2]:
consumer_key = 'grb5AnK0mOzm6BsU7NVbG0Lz3'
consumer_secret = 'thAUHH3fns0TE2wYAGFhC6PrUQktpFO4IAIZWDFeDPvickJAKD'
access_token = '853426630483726336-PtJ837ZcRxu8jxzMS5WJ1ukDaNLp2NM'
access_secret = 'ySMHkMbjXXkDhXpQWpMJX9CThtGBTOKInVMIW0LS9UirN'
oauth = OAuth(access_token, access_secret, consumer_key, consumer_secret)
twitter = Twitter(auth=oauth)
path_local='/Users/lisha/Desktop/Final/Data/Data_Raw/'

##### Cause twitter only return data from the past 7 days, I collect data everyday for the recent tweets. 
##### 100 tweets per .json file
#####  The keywords I use is mainly about the recent UA3411 incident

In [4]:
def date_range(start, end):
    r = (end+datetime.timedelta(days=1)-start).days
    return [start+datetime.timedelta(days=i) for i in range(r)] 
end = datetime.date(2017, 4, 20)
start = end-timedelta(days=0)
dateList = date_range(start, end)

In [5]:
dateList

[datetime.date(2017, 4, 20)]

In [6]:
list_key=['airline','united airline', 'overbook','3411','david dao','hare airport', 'oscar munoz','customer','flight 3411','CNN','louisville international airport','unwilling passenger','apologiz','voucher','passenger','seat','weibo','asian','beat','lawsuit','UA3411','fare','flight','compensation','violent']

In [7]:
for searchquery in list_key:
    for i in dateList:
        t=twitter.search.tweets(q= searchquery, result_type='mixed', lang='en', count=100, until=i)
        timestamp=int(time.mktime(time.strptime(str(datetime.datetime.now().replace(microsecond=0)), '%Y-%m-%d %H:%M:%S')))
        stamp = t['search_metadata']['query']+str(timestamp)
        if t['statuses'] != []:
            f = open(path_local + str(stamp) + '.json',"w")
            json.dump(t,f)
            f.close()
        time.sleep(2)

### 2. Storing Data

In [9]:
list_jsonfiles = glob.glob(path_local + '*.json')

In [10]:
data_dic = {}
for file in list_jsonfiles:
    with open(file) as data_file:
        data_dic[file] = json.load(data_file)

##### To make sure there is no data overwrite, I saved each .json file with its query and an unix timestamp
##### I catagorize the .json files in folders named by its keywords and they will be catagorized by time

In [11]:
for directory in list_jsonfiles:
    path_country = ''
    path_city = ''
    path_term = ''
    path_key = 'Data/Data_Processed/' + directory.split('+')[0].split('/')[-1] + '/'
    path_date = path_key + directory[-25:-15] + '/'
    if not os.path.exists(path_key):
        os.makedirs(path_key)
    if not os.path.exists(path_date):
        os.makedirs(path_date)

In [12]:
for directory in list_jsonfiles:
    dst = ''
    dst = 'Data/Data_Processed/' + directory.split('+')[0].split('/')[-1] + '/' + directory[-25:-15] + '/'
    shutil.copy(directory, dst)

### 3. Preprocess Data
##### Extract all the tweets I collect from the raw data and store them as a list in a .json file

In [9]:
list_tweets=[]
for file in list_jsonfiles:
    with open(file) as data_file:
        d=json.load(data_file)
        tweets=d['statuses']
        list_temp=list(map(lambda tweet: tweet['text'], tweets))
    list_tweets.extend(list_temp)
f = open('/Users/lisha/Desktop/Final/Data/Data_Clean/Tweets.txt',"w")
json.dump(list_tweets,f)
f.close()

In [141]:
list_tweets[:5]

['United CEO response to United Express Flight 3411. https://t.co/rF5gNIvVd0',
 'United CEO Oscar Munoz: I’m sorry. We will fix this. https://t.co/v8EPGsiDCi https://t.co/eOPiYcagvo',
 '@USAnonymous Flight 3411 from Chicago to Louisville was overbooked. After our team looked for volunteers, one customer refused to leave ^MD',
 'RT @UnitedOverBooks: now boarding flight 3411 jk lol we over booked https://t.co/cT2IHKNuXe',
 'RT @UnitedOverBooks: now boarding flight 3411 jk lol we over booked https://t.co/cT2IHKNuXe']

##### Get the information of users info, tweets, location, country and other data I will use in the future analysis and store them in a .csv file

In [128]:
def get_info(tweets):
    df = pd.DataFrame()
    df['id']= list(map(lambda tweet: tweet['id'], tweets)) 
    df['name']=list(map(lambda tweet: tweet['user']['name'], tweets))
    df['created_at']= list(map(lambda tweet: tweet['created_at'], tweets)) 
    df['text'] = list(map(lambda tweet: tweet['text'], tweets))  
    df['country_code'] = list(map(lambda tweet: tweet['place']['country_code'] if tweet['place'] is not None else '', tweets)) 
    df['long'] = list(map(lambda tweet: tweet['place']['bounding_box']['coordinates'][0][0][0] if tweet['place'] is not None else '', tweets)) 
    df['latt'] = list(map(lambda tweet: tweet['place']['bounding_box']['coordinates'][0][0][1]if tweet['place'] is not None else '', tweets))
    df['retweet_count']= list(map(lambda tweet: tweet['retweet_count'], tweets))        
    df['user_created']=list(map(lambda tweet: tweet['user']['created_at'], tweets)) 
    df['followers_count']=list(map(lambda tweet: tweet['user']['followers_count'], tweets))
    df['friends_count'] = list(map(lambda tweet: tweet['user']['friends_count'], tweets))
    df['user_language']=list(map(lambda tweet: tweet['user']['lang'], tweets))
    df['favorite_count']=list(map(lambda tweet: tweet['favorite_count'], tweets)) 
    return df

In [129]:
df=pd.DataFrame()
list_df=[]
for file in list_jsonfiles:
    with open(file) as data_file:
        df_temp=pd.DataFrame()
        d=json.load(data_file)
        tweets=d['statuses']
        df_temp=get_info(tweets)
        list_df.append(df_temp)

In [130]:
df=pd.concat(list_df)

In [131]:
df.head()

,id,name,created_at,text,country_code,long,latt,retweet_count,user_created,followers_count,friends_count,user_language,favorite_count
0,851471781827420160,United,Mon Apr 10 16:27:47 +0000 2017,United CEO response to United Express Flight 3...,US,-95.8233,29.5223,21757,Fri Mar 04 21:23:11 +0000 2011,913424,46648,en,7840
1,851875102769721344,United,Tue Apr 11 19:10:26 +0000 2017,United CEO Oscar Munoz: I’m sorry. We will fix...,US,-95.8233,29.5223,3098,Fri Mar 04 21:23:11 +0000 2011,913424,46648,en,4774
2,851383383888285696,United,Mon Apr 10 10:36:31 +0000 2017,@USAnonymous Flight 3411 from Chicago to Louis...,,,,1373,Fri Mar 04 21:23:11 +0000 2011,913424,46648,en,852
3,851947964599648256,Triston,Tue Apr 11 23:59:57 +0000 2017,RT @UnitedOverBooks: now boarding flight 3411 ...,,,,2235,Sun Jan 01 02:36:00 +0000 2012,1215,987,en,0
4,851947959851921408,TΞddy Rosenthal,Tue Apr 11 23:59:56 +0000 2017,RT @UnitedOverBooks: now boarding flight 3411 ...,,,,2235,Thu Jul 09 15:01:43 +0000 2015,80,227,en,0


In [191]:
df.to_csv('/Users/lisha/Desktop/Final/Data/Data_Clean/Tweets_withINFO.csv')